In [85]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import json

#Set Display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.width' , 1000)
pd.set_option('display.max_columns', None)


In [86]:
# Variables
season = "2020-21"

In [87]:
all_players = pd.read_csv (r'../data/' + season + '/Raw_data/all_players.csv', parse_dates= ["news_added"])
all_teams = pd.read_csv (r'../data/' + season + '/Raw_data/all_teams.csv')
all_events = pd.read_csv (r'../data/' + season + '/Raw_data/all_events.csv', parse_dates= ["deadline_time"])
player_types = pd.read_csv(r'../data/' + season + '/Raw_data/player_types.csv')

understat_player = pd.read_csv(r'../data/' + season + '/Raw_data/understat/players/understat_player.csv')
mapping_understat = pd.read_excel(r'../data/mapping_tables.xlsx' , sheet_name = "understat")


player_season_history = pd.read_csv(r'../data/' + season + '/Raw_data/player_season_history.csv')
player_gameweek_history = pd.read_csv(r'../data/' + season + '/Raw_data/player_gameweek_history.csv')
player_future_fixture = pd.read_csv(r'../data/' + season + '/Raw_data/player_future_fixture.csv', parse_dates = ["kickoff_time"])



## RENAME COLUMNS

In [88]:
mapping_understat.rename(
    columns = {
        "fpl_id" : "player_id"
    },
    inplace = True
)

player_gameweek_history.rename(
    columns = {
        "element" : "player_id" , 
        "opponent_team" : "opponent_team_id" , 
        "fixture" : "fixture_id" ,
        "minutes" : "minutes_played" ,
        "was_home" : "home_game" , 
        "value" : "player_price"
        }, 
        inplace = True
)

all_players.rename(
    columns = {
        "code" : "player_code", 
        "element_type" : "player_type_id" , 
        "minutes" : "total_minutes" , 
        "id" : "player_id" , 
        "team" : "team_id" , 
        "ep_next" : "expected_points_next_gw" ,
        "ep_this" : "expected_points_this_gw" ,
        "news" : "player_news" ,
        "now_cost" : "player_price" ,
        "transfers_in" : "total_transfers_in" ,
        "transfers_out" : "total_transfers_out" ,
        "photo" : "player_photo"
        }, 
        inplace=True
)

all_teams.rename(
    columns = {
        "id" : "team_id" , 
        "code" : "team_code" , 
        "name" : "team_name" , 
        "short_name" : "team_short_name" , 
        "strength" : "team_strength" , 
        "form" : "team_form"
        }, 
        inplace = True
)

all_events.rename(
    columns = {
        "id" : "gameweek_id"
        }, 
        inplace = True
)

player_types.rename(
    columns = {
        "id" : "player_type_id" , 
        "singular_name" : "position_name" , 
        "singular_name_short" : "position_name_short"
        }, 
        inplace = True
)

player_season_history.rename(
    columns = {
        "element_code" : "player_code"
        }, 
        inplace = True
)

player_future_fixture.rename(
    columns = {
        "id" : "fixture_id" , 
        "team_h" : "home_team_id" , 
        "team_a" : "away_team_id" , 
        "event" : "round" , 
        "code" : "fixture_code" ,
        "is_home" : "home_game" 
        }, 
        inplace = True
)

understat_player.rename(
    columns= {
        "player_name" : "understat_player_name" ,
        "id" : "understat_id" ,
        "games" : "games_played" ,
        "time" : "understat_total_minutes" ,
        "assists" : "understat_assists"
         },
         inplace= True
)
        

## Change columns data types

In [89]:
all_players['news_added'] = all_players['news_added'].dt.tz_localize(None)
all_events['deadline_time'] = all_events['deadline_time'].dt.tz_localize(None)
player_future_fixture['kickoff_time'] = player_future_fixture['kickoff_time'].dt.tz_localize(None)

player_gameweek_history['kickoff_time'] = pd.to_datetime(player_gameweek_history['kickoff_time'])
player_gameweek_history['kickoff_time'] = player_gameweek_history['kickoff_time'].dt.tz_localize(None)
player_gameweek_history['total_points'] = player_gameweek_history['total_points'].astype(float)

## ADD COLUMNS

In [90]:
def opponent_team(row):
    if row['home_game'] == True:
        val = row['away_team_id']
    elif row['home_game'] == False:
        val = row['home_team_id']
    return val

def team(row):
    if row['home_game'] == False:
        val = row['away_team_id']
    elif row['home_game'] == True:
        val = row['home_team_id']
    return val

def team_score(row):
    if row['home_game'] == False:
        val = row['team_a_score']
    elif row['home_game'] == True:
        val = row['team_h_score']
    return val

def opponent_team_score(row):
    if row['home_game'] == True:
        val = row['team_a_score']
    elif row['home_game'] == False:
        val = row['team_h_score']
    return val 


player_future_fixture['opponent_team_id'] = player_future_fixture.apply(opponent_team,axis=1)
player_future_fixture['team_id'] = player_future_fixture.apply(team,axis=1)

player_gameweek_history['team_score'] = player_gameweek_history.apply(team_score,axis=1)
player_gameweek_history['opponent_team_score'] = player_gameweek_history.apply(opponent_team_score,axis=1)
player_gameweek_history['player_price'] = player_gameweek_history['player_price']/10


all_players['full_name'] = all_players['first_name'] + ' ' + all_players['second_name']
all_players['player_price'] = all_players['player_price']/10


## DROP COLUMNS

In [91]:
player_types.drop(
    columns = [
        "plural_name" , 
        "plural_name_short" ,
        "ui_shirt_specific" ,
        "sub_positions_locked"
        ], 
        inplace = True
)
        
all_players.drop(
    columns = [
        "squad_number" , 
        "special" ,
        "cost_change_event_fall" ,
        "cost_change_start_fall" ,
        "dreamteam_count" ,
        "in_dreamteam" ,
        "corners_and_indirect_freekicks_text" ,
        "direct_freekicks_text" ,
        "penalties_text" ,
        "first_name" , 
        "second_name",
        "player_code"
        ], 
        inplace = True
)

all_teams.drop(
    columns = [
        "draw" ,
        "team_form" ,
        "loss" ,
        "played" ,
        "points" ,
        "position" ,
        "win" ,
        "team_division"
    ],
    inplace= True
)

understat_player.drop(
    columns= [
        "position" ,
        "team_title" ,
        "yellow_cards" ,
        "red_cards"
    ],
    inplace = True
)

player_gameweek_history.drop(
    columns= [
        "team_h_score" ,
        "team_a_score"
    ],
    inplace = True
)

player_future_fixture.drop(
    columns= [
        "home_team_id" ,
        "away_team_id" ,
        "team_h_score" ,
        "team_a_score" ,
        "finished" ,
        "minutes" ,
        "provisional_start_time" ,
        "fixture_code"
    ],
    inplace = True
)

In [92]:
#player_future_fixture.head()
#player_gameweek_history.head()
all_players.head()
#all_teams.head()
#all_events.head()
#player_types.head()
#understat_player.head()

,chance_of_playing_next_round,chance_of_playing_this_round,cost_change_event,cost_change_start,player_type_id,expected_points_next_gw,expected_points_this_gw,event_points,form,player_id,player_news,news_added,player_price,player_photo,points_per_game,selected_by_percent,status,team_id,team_code,total_points,total_transfers_in,transfers_in_event,total_transfers_out,transfers_out_event,value_form,value_season,web_name,total_minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,direct_freekicks_order,penalties_order,full_name
0,0.0,0.0,0,-2,3,0.0,0.0,0,0.0,1,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20 22:30:18.118477,6.8,37605.jpg,0.0,0.6,n,1,3,0,3312,8,50301,1148,0.0,0.0,Özil,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,600,238,600,238,600,238,600,238,NaN,NaN,NaN,Mesut Özil
1,0.0,0.0,0,-1,2,0.0,0.0,0,0.0,2,Not included in Arsenal's 25-man Premier Leagu...,2020-10-21 10:30:18.546407,4.9,39476.jpg,0.0,0.1,n,1,3,0,10213,8,18292,204,0.0,0.0,Sokratis,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,555,209,546,209,534,203,557,209,NaN,NaN,NaN,Sokratis Papastathopoulos
2,100.0,100.0,0,0,2,0.8,0.8,0,0.3,3,NaN,2020-10-25 21:30:13.186040,5.5,41270.jpg,1.4,0.9,a,1,3,7,34055,2625,67891,2775,0.1,1.3,David Luiz,319,0,0,0,5,0,0,0,0,0,0,0,51,66.6,9.8,21.0,9.7,200,83,309,101,236,70,268,90,NaN,4.0,NaN,David Luiz Moreira Marinho
3,100.0,100.0,0,-3,3,4.5,4.5,2,4.0,4,NaN,2020-10-08 09:00:12.305506,11.7,54694.jpg,3.8,13.0,a,1,3,30,427782,19641,2656471,81719,0.3,2.6,Aubameyang,716,2,1,2,10,0,0,0,2,0,0,1,111,133.4,147.8,151.0,43.2,79,27,32,25,44,25,39,21,NaN,6.0,1.0,Pierre-Emerick Aubameyang
4,100.0,100.0,0,-3,2,0.5,0.5,0,0.0,5,NaN,2020-09-23 09:00:14.881983,4.7,58822.jpg,0.0,0.2,a,1,3,0,2946,134,31354,600,0.0,0.0,Cédric,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,533,203,521,203,498,193,535,203,NaN,NaN,NaN,Cédric Soares


In [93]:
mapping_understat.head()

,player_id,understat_id,first_name,second_name
0,118,702,Tammy,Abraham
1,377,7700,Che,Adams
2,572,5590,Tosin,Adarabioyo
3,268,619,Sergio,Agüero
4,562,725,Ola,Aina


In [94]:
understat_player.head()

,understat_id,understat_player_name,games_played,understat_total_minutes,goals,xG,understat_assists,xA,shots,key_passes,npg,npxG,xGChain,xGBuildup
0,453,Son Heung-Min,8,656,8,3.503271,2,1.804271,18,17,8,3.503271,6.231780,2.453945
1,755,Jamie Vardy,7,559,8,7.259719,1,0.650218,16,4,3,2.692706,3.755111,0.412187
2,1250,Mohamed Salah,8,720,8,5.939518,0,2.035087,34,21,4,2.894843,6.851420,2.755957
3,5555,Dominic Calvert-Lewin,8,703,8,5.818061,0,0.352826,26,4,8,5.818061,5.726510,1.278623
4,647,Harry Kane,8,717,7,6.807162,8,3.798403,38,18,5,5.284825,7.708270,1.147685


## MERGE DATA
## ALL PLAYERS

### all_players & understat_player

In [95]:
df_merge = mapping_understat[["player_id" ,"understat_id"]]
all_players = all_players.merge(df_merge , on = "player_id")
#all_players.head()

df_merge = understat_player[["understat_id" , "games_played" , "xG" , "understat_assists" , "xA" , "shots" , "key_passes" , "npg" , "npxG" , "xGChain" , "xGBuildup"]]
all_players = all_players.merge(df_merge, on= "understat_id")
all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,cost_change_event,cost_change_start,player_type_id,expected_points_next_gw,expected_points_this_gw,event_points,form,player_id,player_news,news_added,player_price,player_photo,points_per_game,selected_by_percent,status,team_id,team_code,total_points,total_transfers_in,transfers_in_event,total_transfers_out,transfers_out_event,value_form,value_season,web_name,total_minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,direct_freekicks_order,penalties_order,full_name,understat_id,games_played,xG,understat_assists,xA,shots,key_passes,npg,npxG,xGChain,xGBuildup
0,100.0,100.0,0,0,2,0.8,0.8,0,0.3,3,NaN,2020-10-25 21:30:13.186040,5.5,41270.jpg,1.4,0.9,a,1,3,7,34055,2625,67891,2775,0.1,1.3,David Luiz,319,0,0,0,5,0,0,0,0,0,0,0,51,66.6,9.8,21.0,9.7,200,83,309,101,236,70,268,90,NaN,4.0,NaN,David Luiz Moreira Marinho,1676,5,0.081089,0,0.000000,2,0,0,0.081089,0.741946,0.741946
1,100.0,100.0,0,-3,3,4.5,4.5,2,4.0,4,NaN,2020-10-08 09:00:12.305506,11.7,54694.jpg,3.8,13.0,a,1,3,30,427782,19641,2656471,81719,0.3,2.6,Aubameyang,716,2,1,2,10,0,0,0,2,0,0,1,111,133.4,147.8,151.0,43.2,79,27,32,25,44,25,39,21,NaN,6.0,1.0,Pierre-Emerick Aubameyang,318,8,1.351223,1,1.316097,10,10,1,0.590054,2.672929,0.848487
2,NaN,NaN,0,-1,4,2.2,2.2,2,1.7,6,NaN,NaT,8.4,59966.jpg,3.9,3.2,a,1,3,27,264421,4870,368248,18338,0.2,3.2,Lacazette,485,3,0,2,5,0,0,0,1,0,0,3,88,126.4,73.8,186.0,38.3,97,14,110,14,34,18,50,16,NaN,5.0,3.0,Alexandre Lacazette,3277,7,2.702362,0,0.309107,9,6,3,2.702362,3.205895,0.328369
3,100.0,100.0,0,0,2,1.2,1.2,0,0.7,7,NaN,2020-08-15 10:00:11.331812,5.0,69140.jpg,1.0,0.0,a,1,3,2,10844,283,10708,188,0.1,0.4,Mustafi,44,0,0,0,1,0,0,0,0,0,0,0,13,11.6,1.0,0.0,1.3,358,140,369,141,554,210,390,145,NaN,NaN,NaN,Shkodran Mustafi,1699,2,0.000000,0,0.000000,0,0,0,0.000000,0.064725,0.064725
4,NaN,NaN,0,0,1,4.2,4.2,2,3.7,8,NaN,NaT,5.0,80201.jpg,3.5,9.0,a,1,3,28,375612,7585,346656,24258,0.7,5.6,Leno,720,0,0,2,10,0,0,0,0,0,21,2,150,171.2,0.0,0.0,17.1,41,8,564,59,556,59,177,9,NaN,NaN,NaN,Bernd Leno,181,8,0.000000,0,0.000000,0,0,0,0.000000,0.880429,0.880429


### Players not mapped

In [96]:
missing_players = mapping_understat.merge(understat_player, on= "understat_id", how = 'right')
missing_players = missing_players[missing_players.player_id.isna()]
missing_players.head()

,player_id,understat_id,first_name,second_name,understat_player_name,games_played,understat_total_minutes,goals,xG,understat_assists,xA,shots,key_passes,npg,npxG,xGChain,xGBuildup


### all_players & all_teams

In [97]:
all_teams.head()

,team_code,team_id,team_name,team_short_name,team_strength,unavailable,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,1,Arsenal,ARS,4,False,1190,1210,1170,1210,1190,1200,1
1,7,2,Aston Villa,AVL,3,False,1150,1160,1150,1150,1180,1210,2
2,36,3,Brighton,BHA,3,False,1080,1100,1150,1180,1090,1100,131
3,90,4,Burnley,BUR,2,False,1050,1080,1120,1190,1010,1030,43
4,8,5,Chelsea,CHE,4,False,1260,1280,1240,1280,1270,1310,4


In [98]:
df_merge = all_teams[["team_id" , "team_name" , "team_short_name" , "team_strength"]]
all_players = all_players.merge(df_merge , on ="team_id")
all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,cost_change_event,cost_change_start,player_type_id,expected_points_next_gw,expected_points_this_gw,event_points,form,player_id,player_news,news_added,player_price,player_photo,points_per_game,selected_by_percent,status,team_id,team_code,total_points,total_transfers_in,transfers_in_event,total_transfers_out,transfers_out_event,value_form,value_season,web_name,total_minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,direct_freekicks_order,penalties_order,full_name,understat_id,games_played,xG,understat_assists,xA,shots,key_passes,npg,npxG,xGChain,xGBuildup,team_name,team_short_name,team_strength
0,100.0,100.0,0,0,2,0.8,0.8,0,0.3,3,NaN,2020-10-25 21:30:13.186040,5.5,41270.jpg,1.4,0.9,a,1,3,7,34055,2625,67891,2775,0.1,1.3,David Luiz,319,0,0,0,5,0,0,0,0,0,0,0,51,66.6,9.8,21.0,9.7,200,83,309,101,236,70,268,90,NaN,4.0,NaN,David Luiz Moreira Marinho,1676,5,0.081089,0,0.000000,2,0,0,0.081089,0.741946,0.741946,Arsenal,ARS,4
1,100.0,100.0,0,-3,3,4.5,4.5,2,4.0,4,NaN,2020-10-08 09:00:12.305506,11.7,54694.jpg,3.8,13.0,a,1,3,30,427782,19641,2656471,81719,0.3,2.6,Aubameyang,716,2,1,2,10,0,0,0,2,0,0,1,111,133.4,147.8,151.0,43.2,79,27,32,25,44,25,39,21,NaN,6.0,1.0,Pierre-Emerick Aubameyang,318,8,1.351223,1,1.316097,10,10,1,0.590054,2.672929,0.848487,Arsenal,ARS,4
2,NaN,NaN,0,-1,4,2.2,2.2,2,1.7,6,NaN,NaT,8.4,59966.jpg,3.9,3.2,a,1,3,27,264421,4870,368248,18338,0.2,3.2,Lacazette,485,3,0,2,5,0,0,0,1,0,0,3,88,126.4,73.8,186.0,38.3,97,14,110,14,34,18,50,16,NaN,5.0,3.0,Alexandre Lacazette,3277,7,2.702362,0,0.309107,9,6,3,2.702362,3.205895,0.328369,Arsenal,ARS,4
3,100.0,100.0,0,0,2,1.2,1.2,0,0.7,7,NaN,2020-08-15 10:00:11.331812,5.0,69140.jpg,1.0,0.0,a,1,3,2,10844,283,10708,188,0.1,0.4,Mustafi,44,0,0,0,1,0,0,0,0,0,0,0,13,11.6,1.0,0.0,1.3,358,140,369,141,554,210,390,145,NaN,NaN,NaN,Shkodran Mustafi,1699,2,0.000000,0,0.000000,0,0,0,0.000000,0.064725,0.064725,Arsenal,ARS,4
4,NaN,NaN,0,0,1,4.2,4.2,2,3.7,8,NaN,NaT,5.0,80201.jpg,3.5,9.0,a,1,3,28,375612,7585,346656,24258,0.7,5.6,Leno,720,0,0,2,10,0,0,0,0,0,21,2,150,171.2,0.0,0.0,17.1,41,8,564,59,556,59,177,9,NaN,NaN,NaN,Bernd Leno,181,8,0.000000,0,0.000000,0,0,0,0.000000,0.880429,0.880429,Arsenal,ARS,4


### all_players & player_types

In [99]:
df_merge = player_types[["player_type_id" , "position_name" , "position_name_short"]]
all_players = all_players.merge(df_merge , on="player_type_id")
all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,cost_change_event,cost_change_start,player_type_id,expected_points_next_gw,expected_points_this_gw,event_points,form,player_id,player_news,news_added,player_price,player_photo,points_per_game,selected_by_percent,status,team_id,team_code,total_points,total_transfers_in,transfers_in_event,total_transfers_out,transfers_out_event,value_form,value_season,web_name,total_minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,direct_freekicks_order,penalties_order,full_name,understat_id,games_played,xG,understat_assists,xA,shots,key_passes,npg,npxG,xGChain,xGBuildup,team_name,team_short_name,team_strength,position_name,position_name_short
0,100.0,100.0,0,0,2,0.8,0.8,0,0.3,3,NaN,2020-10-25 21:30:13.186040,5.5,41270.jpg,1.4,0.9,a,1,3,7,34055,2625,67891,2775,0.1,1.3,David Luiz,319,0,0,0,5,0,0,0,0,0,0,0,51,66.6,9.8,21.0,9.7,200,83,309,101,236,70,268,90,NaN,4.0,NaN,David Luiz Moreira Marinho,1676,5,0.081089,0,0.000000,2,0,0,0.081089,0.741946,0.741946,Arsenal,ARS,4,Defender,DEF
1,100.0,100.0,0,0,2,1.2,1.2,0,0.7,7,NaN,2020-08-15 10:00:11.331812,5.0,69140.jpg,1.0,0.0,a,1,3,2,10844,283,10708,188,0.1,0.4,Mustafi,44,0,0,0,1,0,0,0,0,0,0,0,13,11.6,1.0,0.0,1.3,358,140,369,141,554,210,390,145,NaN,NaN,NaN,Shkodran Mustafi,1699,2,0.000000,0,0.000000,0,0,0,0.000000,0.064725,0.064725,Arsenal,ARS,4,Defender,DEF
2,NaN,NaN,0,1,2,5.2,5.2,1,4.7,11,NaN,NaT,5.1,98745.jpg,4.2,8.2,a,1,3,34,490328,38775,252524,24700,0.9,6.7,Bellerín,720,0,3,2,10,0,0,0,3,0,0,6,152,126.6,91.4,49.0,26.8,96,34,80,21,161,38,93,21,NaN,NaN,NaN,Héctor Bellerín,492,8,0.086247,2,0.625473,2,4,0,0.086247,2.223799,2.052540,Arsenal,ARS,4,Defender,DEF
3,0.0,NaN,0,-2,2,0.0,0.5,0,0.0,14,Illness - Expected back 29 Nov,2020-11-18 14:00:19.656655,4.8,111457.jpg,2.0,0.2,i,1,3,2,4711,140,16107,700,0.0,0.4,Kolasinac,90,0,0,0,1,0,0,0,0,0,0,0,9,3.6,5.4,4.0,1.3,389,148,320,107,326,117,389,144,NaN,NaN,NaN,Sead Kolasinac,342,1,0.000000,0,0.000000,0,0,0,0.000000,0.429358,0.429358,Arsenal,ARS,4,Defender,DEF
4,NaN,NaN,0,-2,2,0.8,0.8,0,0.3,15,NaN,NaT,4.8,154043.jpg,2.2,1.1,a,1,3,9,46535,577,118487,4646,0.1,1.9,Maitland-Niles,187,0,0,1,3,0,0,0,0,0,0,0,35,20.2,4.3,7.0,3.2,326,133,329,115,302,102,357,128,NaN,NaN,NaN,Ainsley Maitland-Niles,1750,4,0.046203,0,0.000000,1,0,0,0.046203,0.627582,0.581378,Arsenal,ARS,4,Defender,DEF


## PLAYER GAMEWEEK HISTORY

### APPEND player_future_gameweek to player_gameweek_history

In [100]:
#player_gameweek_all = player_gameweek_history.append(player_future_fixture)

### player_gameweek_history & all_players

In [101]:
player_gameweek_history.head()

,player_id,fixture_id,opponent_team_id,total_points,home_game,kickoff_time,round,minutes_played,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,player_price,transfers_balance,selected,transfers_in,transfers_out,team_score,opponent_team_score
0,1,2,8,0.0,False,2020-09-12 11:30:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,7.0,0,76656,0,0,3,0
1,1,9,19,0.0,True,2020-09-19 19:00:00,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,6.9,-16828,68335,995,17823,2,1
2,1,23,11,0.0,False,2020-09-28 19:00:00,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,6.9,-11451,59793,675,12126,1,3
3,1,29,15,0.0,True,2020-10-04 13:00:00,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,6.8,-5324,56403,647,5971,2,1
4,1,44,12,0.0,False,2020-10-17 16:30:00,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,6.8,-4224,53689,616,4840,0,1


In [102]:
all_players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'cost_change_event', 'cost_change_start', 'player_type_id', 'expected_points_next_gw', 'expected_points_this_gw', 'event_points', 'form', 'player_id', 'player_news', 'news_added', 'player_price', 'player_photo', 'points_per_game', 'selected_by_percent', 'status', 'team_id', 'team_code', 'total_points', 'total_transfers_in', 'transfers_in_event', 'total_transfers_out', 'transfers_out_event', 'value_form', 'value_season', 'web_name', 'total_minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank', 'creativity_rank_type', 'threat_rank', 'threat_rank_type', 'ict_index_rank', 'ict_index_rank_type', 'corners_and_indirect_freekicks_order', 'direct_freekicks_order', 'penalties_order', 'full_name',
 

In [103]:
df_merge = all_players[["player_id" , "team_id" , "corners_and_indirect_freekicks_order" , "direct_freekicks_order" , "penalties_order" , "team_short_name" , "position_name_short" , "team_strength" , "full_name"]]
player_gameweek_history = player_gameweek_history.merge(df_merge , on = "player_id")

player_gameweek_history.tail()

,player_id,fixture_id,opponent_team_id,total_points,home_game,kickoff_time,round,minutes_played,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,player_price,transfers_balance,selected,transfers_in,transfers_out,team_score,opponent_team_score,team_id,corners_and_indirect_freekicks_order,direct_freekicks_order,penalties_order,team_short_name,position_name_short,team_strength,full_name
3126,590,56,7,1.0,True,2020-10-25 14:00:00,6,1,0,0,0,0,0,0,0,0,0,0,0,3,0.0,0.0,0.0,0.0,4.5,0,0,0,0,2,0,16,NaN,NaN,NaN,SOU,FWD,3,Daniel N'Lundulu
3127,590,59,2,0.0,False,2020-11-01 12:00:00,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,4.5,2506,3338,3037,531,4,3,16,NaN,NaN,NaN,SOU,FWD,3,Daniel N'Lundulu
3128,590,76,14,0.0,True,2020-11-06 20:00:00,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,4.5,30,3431,896,866,2,0,16,NaN,NaN,NaN,SOU,FWD,3,Daniel N'Lundulu
3129,597,67,17,1.0,False,2020-11-01 19:15:00,7,90,0,0,0,2,0,0,0,0,0,2,0,12,19.0,0.0,0.0,1.9,4.5,0,0,0,0,1,2,3,NaN,NaN,NaN,BHA,GKP,3,Robert Sánchez
3130,597,70,4,0.0,True,2020-11-06 17:30:00,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,4.5,1024,1558,1203,179,0,0,3,NaN,NaN,NaN,BHA,GKP,3,Robert Sánchez


### player_gameweek_history & all_teams

In [104]:
all_teams.head()

,team_code,team_id,team_name,team_short_name,team_strength,unavailable,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,1,Arsenal,ARS,4,False,1190,1210,1170,1210,1190,1200,1
1,7,2,Aston Villa,AVL,3,False,1150,1160,1150,1150,1180,1210,2
2,36,3,Brighton,BHA,3,False,1080,1100,1150,1180,1090,1100,131
3,90,4,Burnley,BUR,2,False,1050,1080,1120,1190,1010,1030,43
4,8,5,Chelsea,CHE,4,False,1260,1280,1240,1280,1270,1310,4


In [105]:
df_merge = all_teams[["team_id" , "team_short_name" , "team_strength"]]
df_merge.rename(
    columns= {
        "team_id" : "opponent_team_id" ,
        "team_short_name" : "opponent_team" ,
        "team_strength" : "opponent_team_strength"
    },
    inplace = True
)

player_gameweek_history = player_gameweek_history.merge(df_merge , on="opponent_team_id")
player_gameweek_history.head()

,player_id,fixture_id,opponent_team_id,total_points,home_game,kickoff_time,round,minutes_played,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,player_price,transfers_balance,selected,transfers_in,transfers_out,team_score,opponent_team_score,team_id,corners_and_indirect_freekicks_order,direct_freekicks_order,penalties_order,team_short_name,position_name_short,team_strength,full_name,opponent_team,opponent_team_strength
0,3,2,8,0.0,False,2020-09-12 11:30:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,5.5,0,69564,0,0,3,0,1,NaN,4.0,NaN,ARS,DEF,4,David Luiz Moreira Marinho,FUL,2
1,4,2,8,7.0,False,2020-09-12 11:30:00,1,90,1,0,1,0,0,0,0,1,0,0,0,19,36.6,15.3,54.0,10.6,12.0,0,2823465,0,0,3,0,1,NaN,6.0,1.0,ARS,MID,4,Pierre-Emerick Aubameyang,FUL,2
2,6,2,8,7.0,False,2020-09-12 11:30:00,1,86,1,0,1,0,0,0,0,0,0,0,1,29,38.6,12.7,48.0,9.9,8.5,0,196064,0,0,3,0,1,NaN,5.0,3.0,ARS,FWD,4,Alexandre Lacazette,FUL,2
3,7,2,8,0.0,False,2020-09-12 11:30:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,5.0,0,899,0,0,3,0,1,NaN,NaN,NaN,ARS,DEF,4,Shkodran Mustafi,FUL,2
4,8,2,8,7.0,False,2020-09-12 11:30:00,1,90,0,0,1,0,0,0,0,0,0,2,1,29,14.0,0.0,0.0,1.4,5.0,0,400285,0,0,3,0,1,NaN,NaN,NaN,ARS,GKP,4,Bernd Leno,FUL,2


In [106]:
player_future_fixture.head()

,fixture_id,round,kickoff_time,event_name,home_game,difficulty,player_id,opponent_team_id,team_id
0,82,9.0,2020-11-22 16:30:00,Gameweek 9,False,3,1,10,1
1,89,10.0,2020-11-29 19:15:00,Gameweek 10,True,3,1,20,1
2,106,11.0,2020-12-05 15:00:00,Gameweek 11,False,4,1,17,1
3,109,12.0,2020-12-13 15:00:00,Gameweek 12,True,2,1,4,1
4,119,13.0,2020-12-15 19:45:00,Gameweek 13,True,3,1,16,1


In [107]:
df_merge = all_players[["player_id" , "corners_and_indirect_freekicks_order" , "direct_freekicks_order" , "penalties_order" , "team_short_name" , "position_name_short" , "team_strength" , "full_name"]]
player_future_fixture = player_future_fixture.merge(df_merge , on = "player_id")

df_merge = all_teams[["team_id" , "team_short_name" , "team_strength"]]
df_merge.rename(
    columns= {
        "team_id" : "opponent_team_id" ,
        "team_short_name" : "opponent_team" ,
        "team_strength" : "opponent_team_strength"
    },
    inplace = True
)

player_future_fixture = player_future_fixture.merge(df_merge , on="opponent_team_id")
player_future_fixture.head()

,fixture_id,round,kickoff_time,event_name,home_game,difficulty,player_id,opponent_team_id,team_id,corners_and_indirect_freekicks_order,direct_freekicks_order,penalties_order,team_short_name,position_name_short,team_strength,full_name,opponent_team,opponent_team_strength
0,82,9.0,2020-11-22 16:30:00,Gameweek 9,False,3,3,10,1,NaN,4.0,NaN,ARS,DEF,4,David Luiz Moreira Marinho,LEE,3
1,229,24.0,2021-02-13 15:00:00,Gameweek 24,True,2,3,10,1,NaN,4.0,NaN,ARS,DEF,4,David Luiz Moreira Marinho,LEE,3
2,82,9.0,2020-11-22 16:30:00,Gameweek 9,False,3,4,10,1,NaN,6.0,1.0,ARS,MID,4,Pierre-Emerick Aubameyang,LEE,3
3,229,24.0,2021-02-13 15:00:00,Gameweek 24,True,2,4,10,1,NaN,6.0,1.0,ARS,MID,4,Pierre-Emerick Aubameyang,LEE,3
4,82,9.0,2020-11-22 16:30:00,Gameweek 9,False,3,6,10,1,NaN,5.0,3.0,ARS,FWD,4,Alexandre Lacazette,LEE,3


In [108]:
player_gameweek_history.direct_freekicks_order.values

array([ 4.,  6.,  5., ...,  4., nan, nan])

In [109]:
def penalties_order(row):
    if row['penalties_order'] > 2:
        val = 0
    elif row['penalties_order'] != row['penalties_order']:
        val = 0
    else:
        val = row['penalties_order']
    return val

def direct_freekicks_order(row):
    if row['direct_freekicks_order'] > 2:
        val = 0
    elif row['direct_freekicks_order'] != row['direct_freekicks_order']:
        val = 0
    else:
        val = row['direct_freekicks_order']
    return val

def corners_and_indirect_freekicks_order(row):
    if row['corners_and_indirect_freekicks_order'] > 2:
        val = 0
    elif row['corners_and_indirect_freekicks_order'] != row['corners_and_indirect_freekicks_order']:
        val = 0
    else:
        val = row['corners_and_indirect_freekicks_order']
    return val


player_gameweek_history['penalties_order'] = player_gameweek_history.apply(penalties_order,axis=1)
player_gameweek_history['direct_freekicks_order'] = player_gameweek_history.apply(direct_freekicks_order,axis=1)
player_gameweek_history['corners_and_indirect_freekicks_order'] = player_gameweek_history.apply(corners_and_indirect_freekicks_order,axis=1)

player_gameweek_history.head()

,player_id,fixture_id,opponent_team_id,total_points,home_game,kickoff_time,round,minutes_played,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,player_price,transfers_balance,selected,transfers_in,transfers_out,team_score,opponent_team_score,team_id,corners_and_indirect_freekicks_order,direct_freekicks_order,penalties_order,team_short_name,position_name_short,team_strength,full_name,opponent_team,opponent_team_strength
0,3,2,8,0.0,False,2020-09-12 11:30:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,5.5,0,69564,0,0,3,0,1,0.0,0.0,0.0,ARS,DEF,4,David Luiz Moreira Marinho,FUL,2
1,4,2,8,7.0,False,2020-09-12 11:30:00,1,90,1,0,1,0,0,0,0,1,0,0,0,19,36.6,15.3,54.0,10.6,12.0,0,2823465,0,0,3,0,1,0.0,0.0,1.0,ARS,MID,4,Pierre-Emerick Aubameyang,FUL,2
2,6,2,8,7.0,False,2020-09-12 11:30:00,1,86,1,0,1,0,0,0,0,0,0,0,1,29,38.6,12.7,48.0,9.9,8.5,0,196064,0,0,3,0,1,0.0,0.0,0.0,ARS,FWD,4,Alexandre Lacazette,FUL,2
3,7,2,8,0.0,False,2020-09-12 11:30:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,5.0,0,899,0,0,3,0,1,0.0,0.0,0.0,ARS,DEF,4,Shkodran Mustafi,FUL,2
4,8,2,8,7.0,False,2020-09-12 11:30:00,1,90,0,0,1,0,0,0,0,0,0,2,1,29,14.0,0.0,0.0,1.4,5.0,0,400285,0,0,3,0,1,0.0,0.0,0.0,ARS,GKP,4,Bernd Leno,FUL,2


## Save new datasets

In [110]:
player_gameweek_history.to_csv(r'../data/2020-21/ML_dataset/ML_player_gameweek_history.csv' , index = False , header = True)
player_future_fixture.to_csv(r'../data/2020-21/ML_dataset/ML_player_future_fixture.csv' , index = False , header = True)
all_players.to_csv(r'../data/2020-21/ML_dataset/ML_all_players.csv' , index = False , header = True)

## Add average opponent strength and scores:

In [130]:
player_gameweek_history.shape

(3131, 41)

In [131]:
player_gameweek_history.head()

,player_id,fixture_id,opponent_team_id,total_points,home_game,kickoff_time,round,minutes_played,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,player_price,transfers_balance,selected,transfers_in,transfers_out,team_score,opponent_team_score,team_id,corners_and_indirect_freekicks_order,direct_freekicks_order,penalties_order,team_short_name,position_name_short,team_strength,full_name,opponent_team,opponent_team_strength
0,3,2,8,0.0,False,2020-09-12 11:30:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,5.5,0,69564,0,0,3,0,1,0.0,0.0,0.0,ARS,DEF,4,David Luiz Moreira Marinho,FUL,2
1,4,2,8,7.0,False,2020-09-12 11:30:00,1,90,1,0,1,0,0,0,0,1,0,0,0,19,36.6,15.3,54.0,10.6,12.0,0,2823465,0,0,3,0,1,0.0,0.0,1.0,ARS,MID,4,Pierre-Emerick Aubameyang,FUL,2
2,6,2,8,7.0,False,2020-09-12 11:30:00,1,86,1,0,1,0,0,0,0,0,0,0,1,29,38.6,12.7,48.0,9.9,8.5,0,196064,0,0,3,0,1,0.0,0.0,0.0,ARS,FWD,4,Alexandre Lacazette,FUL,2
3,7,2,8,0.0,False,2020-09-12 11:30:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,5.0,0,899,0,0,3,0,1,0.0,0.0,0.0,ARS,DEF,4,Shkodran Mustafi,FUL,2
4,8,2,8,7.0,False,2020-09-12 11:30:00,1,90,0,0,1,0,0,0,0,0,0,2,1,29,14.0,0.0,0.0,1.4,5.0,0,400285,0,0,3,0,1,0.0,0.0,0.0,ARS,GKP,4,Bernd Leno,FUL,2


In [276]:
df = player_gameweek_history.copy()
df = df.sort_values(by='round')
player_id = list(range(1,5))
for id in player_id:
    print(id)
    df = player_gameweek_history
    df = df[df['player_id'] == id]
    df = df[['player_id' ,'round' , 'total_points']]
    df['avg_tot_points'] = df.total_points.rolling(window=4 , win_type='triang').mean().shift()
    df_orig = df_orig.merge(df, how = 'left' , on = ['player_id' , 'round'])
    print(df)
    #print(df[df['round'] == 8])

1
Empty DataFrame
Columns: [player_id, round, total_points, avg_tot_points]
Index: []
2
Empty DataFrame
Columns: [player_id, round, total_points, avg_tot_points]
Index: []
3
      player_id  round  total_points  avg_tot_points
0             3      1           0.0             NaN
155           3      2           1.0             NaN
322           3      3           1.0             NaN
485           3      4           2.0             NaN
643           3      5           2.0           1.000
779           3      6           1.0           1.500
933           3      7           0.0           1.750
1075          3      8           0.0           1.375
4
      player_id  round  total_points  avg_tot_points
1             4      1           7.0             NaN
156           4      2           5.0             NaN
323           4      3           2.0             NaN
486           4      4           2.0             NaN
644           4      5           2.0           3.750
780           4      6       